# Traitement de la source de données

In [2]:
import os
from datetime import date, timedelta

import pandas as pd

In [3]:
# Racine des fichiers quotidiens
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'

# Dates de disponibilité des fichiers
START_DATE = date(2020, 1, 22)
END_DATE = date(2020, 3, 13)

# Répertoire de sauvegarde des fichiers bruts
RAWFILES_DIR = '../data/raw/'
PROCESSED_DIR = '../data/processed/'

# Fichier principal
ALL_DATA_FILE = 'all_data.csv'

#TODO: A remplacer par la lecture du fichier env.yaml

## Boucle de récupération des fichiers

In [4]:
delta = END_DATE - START_DATE       # as timedelta

for i in range(delta.days + 1):
    day = START_DATE + timedelta(days=i)
    day_label = day.strftime("%m-%d-%Y")
    #print(day_label)
    virus_df = pd.read_csv(BASE_URL.format(day_label), sep=',', parse_dates=['Last Update'])
    virus_df.to_csv(os.path.join(RAWFILES_DIR, day_label + '.csv'), index=False)


## Constitution de la table de références lat / log

In [5]:
import glob

df_list = []

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',')
    if 'Latitude' in virus_df.columns and 'Longitude' in virus_df.columns:
        df_list.append(virus_df)

all_df = pd.concat(df_list)

# Création d'une table de références pour les lat/long
(all_df[['Province/State', 'Country/Region', 'Latitude', 'Longitude']]
 .drop_duplicates(subset=['Province/State', 'Country/Region'])
 .sort_values(by=['Country/Region', 'Province/State'])
 .to_csv(os.path.join(PROCESSED_DIR, 'lat_long_table.csv'), index=False)
)

## Construction d'une table unique

In [6]:
data_catalog = {
    'Last Update': ['<M8[ns]'],
    'Confirmed': ['float64', 'int64'],
    'Deaths': ['float64', 'int64'],
    'Recovered': ['float64', 'int64'],
    'Latitude': ['float64'],
    'Longitude': ['float64'],
}

In [7]:
df_list = []
latlong_df = pd.read_csv(os.path.join(PROCESSED_DIR, 'lat_long_table.csv'))

# Lecture des fichiers récupérés et sélection de ceux qui ont une lat / long
for file in glob.glob(os.path.join(RAWFILES_DIR, '*.csv')):
    virus_df = pd.read_csv(file, sep=',', parse_dates=['Last Update'])
    if not('Latitude' in virus_df.columns and 'Longitude' in virus_df.columns):
        virus_df = virus_df.merge(latlong_df, on=['Province/State', 'Country/Region'], how='left')
        
    for field, types in data_catalog.items():
        assert virus_df[field].dtypes in types, f"Bad type for {field} in {file}"
        
    df_list.append(virus_df.assign(source=os.path.basename(file)))

all_df = pd.concat(df_list)

# Sauvegarde de la table totale
all_df.to_csv(os.path.join(PROCESSED_DIR, 'all_data.csv'), index=False)

In [8]:
all_df.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,source
0,Hubei,Mainland China,2020-02-26 14:13:10,65187.0,2615.0,20969.0,30.9756,112.2707,02-26-2020.csv
1,Guangdong,Mainland China,2020-02-26 10:33:02,1347.0,7.0,851.0,23.3417,113.4244,02-26-2020.csv
2,Henan,Mainland China,2020-02-26 10:33:02,1271.0,19.0,1033.0,33.8820,113.6140,02-26-2020.csv
3,NaN,South Korea,2020-02-26 11:03:11,1261.0,12.0,22.0,36.0000,128.0000,02-26-2020.csv
4,Zhejiang,Mainland China,2020-02-26 10:33:02,1205.0,1.0,867.0,29.1832,120.0934,02-26-2020.csv


In [13]:
worldpop = pd.read_csv(os.path.join(RAWFILES_DIR, 'worldpop/worldpop.csv'), delimiter=',')

In [16]:
worldpop

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Population, total",SP.POP.TOTL,5.421100e+04,5.543800e+04,5.622500e+04,5.669500e+04,5.703200e+04,5.736000e+04,...,1.020460e+05,1.025600e+05,1.031590e+05,1.037740e+05,1.043410e+05,1.048720e+05,1.053660e+05,1.058450e+05,NaN,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8.996973e+06,9.169410e+06,9.351441e+06,9.543205e+06,9.744781e+06,9.956320e+06,...,3.011741e+07,3.116138e+07,3.226959e+07,3.337079e+07,3.441360e+07,3.538313e+07,3.629640e+07,3.717239e+07,NaN,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5.454933e+06,5.531472e+06,5.608539e+06,5.679458e+06,5.735044e+06,5.770570e+06,...,2.422066e+07,2.510793e+07,2.601578e+07,2.694178e+07,2.788438e+07,2.884248e+07,2.981675e+07,3.080976e+07,NaN,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1.608800e+06,1.659800e+06,1.711319e+06,1.762621e+06,1.814135e+06,1.864791e+06,...,2.905195e+06,2.900401e+06,2.895092e+06,2.889104e+06,2.880703e+06,2.876101e+06,2.873457e+06,2.866376e+06,NaN,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,1.341100e+04,1.437500e+04,1.537000e+04,1.641200e+04,1.746900e+04,1.854900e+04,...,8.374700e+04,8.242700e+04,8.077400e+04,7.921300e+04,7.801100e+04,7.729700e+04,7.700100e+04,7.700600e+04,NaN,NaN
5,Arab World,ARB,"Population, total",SP.POP.TOTL,9.219775e+07,9.472451e+07,9.733444e+07,1.000342e+08,1.028328e+08,1.057364e+08,...,3.631587e+08,3.714435e+08,3.797057e+08,3.879077e+08,3.960283e+08,4.040244e+08,4.118990e+08,4.197906e+08,NaN,NaN
6,United Arab Emirates,ARE,"Population, total",SP.POP.TOTL,9.241800e+04,1.007960e+05,1.121180e+05,1.251300e+05,1.380390e+05,1.498570e+05,...,8.946777e+06,9.141596e+06,9.197910e+06,9.214175e+06,9.262900e+06,9.360980e+06,9.487203e+06,9.630959e+06,NaN,NaN
7,Argentina,ARG,"Population, total",SP.POP.TOTL,2.048178e+07,2.081727e+07,2.115305e+07,2.148891e+07,2.182442e+07,2.215965e+07,...,4.126149e+07,4.173327e+07,4.220294e+07,4.266950e+07,4.313197e+07,4.359037e+07,4.404481e+07,4.449450e+07,NaN,NaN
8,Armenia,ARM,"Population, total",SP.POP.TOTL,1.874121e+06,1.941492e+06,2.009526e+06,2.077578e+06,2.145001e+06,2.211319e+06,...,2.876538e+06,2.884229e+06,2.897584e+06,2.912403e+06,2.925553e+06,2.936146e+06,2.944809e+06,2.951776e+06,NaN,NaN
9,American Samoa,ASM,"Population, total",SP.POP.TOTL,2.012300e+04,2.060200e+04,2.125300e+04,2.203400e+04,2.285400e+04,2.367200e+04,...,5.575900e+04,5.566700e+04,5.571300e+04,5.579100e+04,5.581200e+04,5.574100e+04,5.562000e+04,5.546500e+04,NaN,NaN


In [15]:
worldpop[['Country Name', 'Country Code', '2020']]

KeyError: "['2020'] not in index"